<a href="https://colab.research.google.com/github/dropcreations/MKVToolNix-in-Google-Colab/blob/main/MKVToolNix-in-Google-Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Mount Google Drive__

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3><b>Mount Google Drive</b></h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

# __MKVToolNix__

* Run below cell to **install** MKVToolNix.

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/2/21/Mkvmerge256.png/64px-Mkvmerge256.png' height="50" alt="mkvtoolnix-logo"/></center>
#@markdown <center><h3><b>Install MKVToolNix</b></h3></center><br>
from IPython.display import clear_output
!sudo curl -L https://mkvtoolnix.download/appimage/MKVToolNix_GUI-66.0.0-x86_64.AppImage -o /usr/local/bin/MKVToolNix_GUI-66.0.0-x86_64.AppImage
!sudo chmod u+rx /usr/local/bin/MKVToolNix_GUI-66.0.0-x86_64.AppImage
!sudo ln -s /usr/local/bin/MKVToolNix_GUI-66.0.0-x86_64.AppImage /usr/local/bin/mkvpropedit
!sudo chmod a+rx /usr/local/bin/mkvpropedit
!sudo ln -s /usr/local/bin/MKVToolNix_GUI-66.0.0-x86_64.AppImage /usr/local/bin/mkvmerge
!sudo chmod a+rx /usr/local/bin/mkvmerge
!sudo ln -s /usr/local/bin/MKVToolNix_GUI-66.0.0-x86_64.AppImage /usr/local/bin/mkvextract
!sudo chmod a+rx /usr/local/bin/mkvextract
!sudo ln -s /usr/local/bin/MKVToolNix_GUI-66.0.0-x86_64.AppImage /usr/local/bin/mkvinfo
!sudo chmod a+rx /usr/local/bin/mkvinfo
clear_output()
print("Successfully Installed.")

### __mkvmerge__

* You can **input tracks** while running the cell.
* Don't use `#` and `"` in `file_title`.
* Add XML file path for `global_tags`, `segment_info`.
* Choose a **split mode** and add split argument **according** to the split mode.
* Chapter are accpeted **both** `XML` and `OGM_txt` files.
* When adding a language use **laguage codes**.
* If you don't want to fill a field, leave it blank.
* If you don't know what is the relevant `mime-type`, also leave it blank in **attachments section**.
* When adding track `default`, `forced` flags type '**Yes**' or '**No**', If you leave it blank, it is value will be '**No**'.

In [ ]:
output_folder = "" #@param {type:"string"}
#@markdown <h3><b>General</b></h3>
file_title = "" #@param {type:"string"}
global_tags = "" #@param {type:"string"}
segment_info = "" #@param {type:"string"}
#@markdown <h3><b>Split</b></h3>
split_mode = "Do not split" #@param ["Do not split", "After output size", "After output duration", "After specific timestamps", "by parts based on timestamps", "by parts based on frame/field numbers", "After frame/field numbers", "Before chapters"]
split_argument = "" #@param {type:"string"}
max_splits = 0 #@param {type:"integer"}
link_files = False #@param {type:"boolean"}
#@markdown <h3><b>Chapters</b></h3>
chapters_file = "" #@param {type:"string"}
chapters_language = "" #@param {type:"string"}
#@markdown <h3><b>Attachments</b></h3>
attach_file = "" #@param {type:"string"}
attachment_name = "" #@param {type:"string"}
attachment_mime_type = "" #@param {type:"string"}
attachment_description = "" #@param {type:"string"}
#@markdown <h3><b>Miscellaneous</b></h3>
webm_compliant_file = False #@param {type:"boolean"}
disable_track_statistics_tags = False #@param {type:"boolean"}

import os
import subprocess
import json

def load_json_out():
    global json_data
    global input_file
    mkvmerge_json_out = subprocess.check_output([
        'mkvmerge',
        '--identify',
        '--identification-format',
        'json',
        input_file,
        ], stderr=subprocess.DEVNULL)
    json_data = json.loads(mkvmerge_json_out)

def load_json_data():
    global id
    global codec_id
    global language
    global language_ietf
    global title
    global codec
    global track_type
    global json_data

    load_json_out()
    id = json_data.get('tracks')[int(i)].get('id')
    language = json_data.get('tracks')[int(i)].get('properties').get('language')
    language_ietf = json_data.get('tracks')[int(i)].get('properties').get('language_ietf')
    title = json_data.get('tracks')[int(i)].get('properties').get('track_name')
    codec_id = json_data.get('tracks')[int(i)].get('properties').get('codec_id')
    codec = json_data.get('tracks')[int(i)].get('codec')
    track_type = json_data.get('tracks')[int(i)].get('type')

def list_tracks():
    global i
    load_json_out()
    track_count = len(json_data['tracks'])
    for i in range(track_count):
        load_json_data()
        print(str(id) + " : " + str(track_type) + " : " + str(language) + " : " + str(codec))

if output_folder == "":
    output_folder = "/content/"

input_file = ""
input_track_params = ""
input_track_info = []
input_file_list = []

while True:
    input_file = input("Insert track: ")
    if input_file.lower() == "done":
        break
    input_file_list.append(input_file)
    load_json_out()
    if len(json_data['tracks']) == 1:
        print("|")
        track_title = input("|--Track Title: ")
        if track_title == "":
            input_track_params = input_track_params
        else:
            input_track_params = input_track_params + '--track_name 0:"' + str(track_title) + '" '
        track_lang = input("|--Track Language: ")
        if track_lang == "":
            input_track_params = input_track_params
        else:
            input_track_params = input_track_params + "--language 0:" + str(track_lang) + " "
        track_default = input("|--Default Track: ")
        if track_default == "" or track_default.lower() == "no":
            input_track_params = input_track_params + "--default-track-flag 0:no "
        elif track_default.lower() == "yes":
            input_track_params = input_track_params + "--default-track-flag 0:yes "
        track_forced = input("|--Forced Track: ")
        if track_forced == "" or track_forced.lower() == "no":
            input_track_params = input_track_params + "--forced-display-flag 0:no "
        elif track_forced == "yes":
            input_track_params = input_track_params + "--forced-display-flag 0:yes "
        print(" ")
        input_track_params = input_track_params + '"' + input_file + '" '
        input_track_info.append(input_track_params)
        input_track_params = ""
    elif len(json_data['tracks']) > 1:
        print(" ")
        print("Available tracks: ")
        list_tracks()
        input_id = input("Enter track ID: ")
        load_json_out()
        if json_data.get('tracks')[int(input_id)].get('type') == "video":
            display_dimensions = json_data.get('tracks')[int(input_id)].get('properties').get('display_dimensions')
            input_track_params = input_track_params + "--no-audio --no-subtitles --no-attachments --no-track-tags --no-global-tags --no-chapters --display-dimensions " + input_id + ":" + display_dimensions + " "
        elif json_data.get('tracks')[int(input_id)].get('type') == "audio":
            input_track_params = input_track_params + "--no-video --no-subtitles --no-attachments --no-track-tags --no-global-tags --no-chapters "
        elif json_data.get('tracks')[int(input_id)].get('type') == "subtitles":
            input_track_params = input_track_params + "--no-video --no-audio --no-attachments --no-track-tags --no-global-tags --no-chapters "
        print("|")
        track_title = input("|--Track Title: ")
        if track_title == "":
            input_track_params = input_track_params
        else:
            input_track_params = input_track_params + "--track-name " + input_id + ':"' + str(track_title) + '" '
        track_lang = input("|--Track Language: ")
        if track_lang == "":
            input_track_params = input_track_params
        else:
            input_track_params = input_track_params + "--language " + input_id + ":" + str(track_lang) + " "
        track_default = input("|--Default Track: ")
        if track_default == "" or track_default.lower() == "no":
            input_track_params = input_track_params + "--default-track-flag " + input_id + ":no "
        elif track_default.lower() == "yes":
            input_track_params = input_track_params + "--default-track-flag " + input_id + ":yes "
        track_forced = input("|--Forced Track: ")
        if track_forced == "" or track_forced.lower() == "no":
            input_track_params = input_track_params + "--forced-display-flag " + input_id + ":no "
        elif track_forced == "yes":
            input_track_params = input_track_params + "--forced-display-flag " + input_id + ":yes "
        print(" ")
        input_track_params = input_track_params + '"' + input_file + '" '
        input_track_info.append(input_track_params)
        input_track_params = ""

if global_tags == "":
    input_track_params = input_track_params
else:
    input_track_params = input_track_params + '--global-tags "' + global_tags + '" '

if segment_info == "":
    input_track_params = input_track_params
else:
    input_track_params = input_track_params + '--segmentinfo "' + segment_info + '" '

if split_mode == "Do not split":
    input_track_params = input_track_params
elif split_mode == "After output size":
    input_track_params = input_track_params + "--split size:" + split_argument + " "
elif split_mode == "After output duration":
    input_track_params = input_track_params + "--split duration:" + split_argument + " "
elif split_mode == "After specific timestamps":
    input_track_params = input_track_params + "--split timestamps:" + split_argument + " "
elif split_mode == "by parts based on timestamps":
    input_track_params = input_track_params + "--split parts:" + split_argument + " "
elif split_mode == "by parts based on frame/field numbers":
    input_track_params = input_track_params + "--split parts-frames:" + split_argument + " "
elif split_mode == "After frame/field numbers":
    input_track_params = input_track_params + "--split frames:" + split_argument + " "
elif split_mode == "Before chapters":
    input_track_params = input_track_params + "--split chapters:" + split_argument + " "

if max_splits == 0 or max_splits == 1:
    input_track_params = input_track_params
elif max_splits > 1:
    input_track_params = input_track_params + "--split-max-files " + str(max_splits) + " "

if link_files == True:
    input_track_params = input_track_params + "--link "
else:
    input_track_params = input_track_params

if chapters_file == "":
    input_track_params = input_track_params
else:
    if chapters_language == "":
        input_track_params = input_track_params + '--chapters "' + chapters_file + '" '
    else:
        input_track_params = input_track_params + "--chapter-language " + chapters_language + ' --chapters "' + chapters_file + '" '

if attach_file == "":
    input_track_params = input_track_params
else:
    if attachment_name == "":
        input_track_params = input_track_params
    else:
        base_name = os.path.splitext(attach_file)
        if attachment_name.endswith(base_name[-1]):
            attachment_name = attachment_name
        else:
            attachment_name = attachment_name + base_name[-1]
        input_track_params = input_track_params + '--attachment-name "' + attachment_name + '" '
    if attachment_mime_type == "":
        input_track_params = input_track_params
    else:
        input_track_params = input_track_params + '--attachment-mime-type ' + attachment_mime_type + " "
    if attachment_description == "":
        input_track_params = input_track_params
    else:
        input_track_params = input_track_params + '--attachment-description "' + attachment_description + '" '
    input_track_params = input_track_params + '--attach-file "' + attach_file + '" '

print(" ")
output_filename = input("Output filename: ")
if output_filename == "":
    if file_title == "":
        in_file = input_file_list[0].split("/")
        out_ext_rem = os.path.splitext(input_file_list[0])
        if webm_compliant_file == True:
            output_filename = in_file[-1][0:len(in_file[-1]) - len(out_ext_rem[-1])] + ".webm"
        else:
            output_filename = in_file[-1][0:len(in_file[-1]) - len(out_ext_rem[-1])] + ".mkv"
    else:
        if webm_compliant_file == True:
            output_filename = file_title + '.webm'
        else:
            output_filename = file_title + '.mkv'
else:
    output_folder = os.path.abspath(output_folder)
    if output_filename.endswith('.webm') or output_filename.endswith('.mkv'):
        output_filename = output_filename
    else:
        if webm_compliant_file == True:
            output_filename = output_filename + '.webm'
        else:
            output_filename = output_filename + '.mkv'
cmd_line = "--output " + '"' + output_folder + "/" + output_filename + '" '

if webm_compliant_file == True:
    cmd_line = cmd_line + '--webm '

if disable_track_statistics_tags == True:
    cmd_line = cmd_line + '--disable-track-statistics-tags '

cmd_line = cmd_line + ''.join(input_track_info)
cmd_line = cmd_line + input_track_params
!mkvmerge {cmd_line}

### __mkvextract__

* You can extract **all tracks, attachments, chapters, tags, cues, cue sheet, timecodes** from below cell.
* Also you can extract a **single track** by selecting `extract_mode: single track` option.
* `chapters_mode` is for `extract_mode: chapters`

In [ ]:
import os
import sys
import subprocess
import json
from IPython.display import clear_output

mkv_file = "" #@param {type:"string"}
extract_mode = "Tracks (All)" #@param ["Tracks (All)", "Chapters", "Attachments", "Tags", "Cue Sheet", "Timecodes", "Cues", "Single Track"]
chapters_mode = "XML" #@param ["XML", "OGM"]

def process_mkv_file():
    global id
    global codec_id
    global language
    global language_ietf
    global title
    global codec
    global track_type
    global extract_track_name

    extract_track_name = str(id) + "_" + str(track_type) + "_" + str(language)

    if "AVC" in codec_id:
        extract_track_name = extract_track_name + ".h264"
    elif "HEVC" in codec_id:
        extract_track_name = extract_track_name + ".h265"
    elif "V_VP8" in codec_id or "V_VP9" in codec_id:
        extract_track_name = extract_track_name + ".ivf"
    elif "V_AV1" in codec_id:
        extract_track_name = extract_track_name + ".ivf"
    elif "V_MPEG1" in codec_id or "V_MPEG2" in codec_id:
        extract_track_name = extract_track_name + ".mpg"
    elif "V_REAL" in codec_id:
        extract_track_name = extract_track_name + ".rm"
    elif "V_THEORA" in codec_id:
        extract_track_name = extract_track_name + ".ogg"
    elif "V_MS/VFW/FOURCC" in codec_id:
        extract_track_name = extract_track_name + ".avi"
    elif "AAC" in codec_id:
        extract_track_name = extract_track_name + ".aac"
    elif "A_AC3" in codec_id:
        extract_track_name = extract_track_name + ".ac3"
    elif "A_EAC3" in codec_id:
        extract_track_name = extract_track_name + ".eac3"
    elif "ALAC" in codec_id:
        extract_track_name = extract_track_name + ".caf"
    elif "DTS" in codec_id:
        extract_track_name = extract_track_name + ".dts"
    elif "FLAC" in codec_id:
        extract_track_name = extract_track_name + ".flac"
    elif "MPEG/L2" in codec_id:
        extract_track_name = extract_track_name + ".mp2"
    elif "MPEG/L3" in codec_id:
        extract_track_name = extract_track_name + ".mp3"
    elif "OPUS" in codec_id:
        extract_track_name = extract_track_name + ".ogg"
    elif "PCM" in codec_id:
        extract_track_name = extract_track_name + ".wav"
    elif "REAL" in codec_id:
        extract_track_name = extract_track_name + ".ra"
    elif "TRUEHD" in codec_id:
        extract_track_name = extract_track_name + ".thd"
    elif "MLP" in codec_id:
        extract_track_name = extract_track_name + ".mlp"
    elif "TTA1" in codec_id:
        extract_track_name = extract_track_name + ".tta"
    elif "VORBIS" in codec_id:
        extract_track_name = extract_track_name + ".ogg"
    elif "WAVPACK4" in codec_id:
        extract_track_name = extract_track_name + ".wv"
    elif "PGS" in codec_id:
        extract_track_name = extract_track_name + ".sup"
    elif "ASS" in codec_id:
        extract_track_name = extract_track_name + ".ass"
    elif "SSA" in codec_id:
        extract_track_name = extract_track_name + ".ssa"
    elif "UTF8" in codec_id or "ASCII" in codec_id:
        extract_track_name = extract_track_name + ".srt"
    elif "VOBSUB" in codec_id:
        extract_track_name = extract_track_name + ".sub"
    elif "S_KATE" in codec_id:
        extract_track_name = extract_track_name + ".ogg"
    elif "USF" in codec_id:
        extract_track_name = extract_track_name + ".usf"
    elif "WEBVTT" in codec_id:
        extract_track_name = extract_track_name + ".vtt"

def execute_command_tracks():
    global id
    global codec_id
    global language
    global language_ietf
    global title
    global codec
    global track_type

    folder_name = os.path.dirname(os.path.abspath(mkv_file))
    folder_mkv, file_mkv = os.path.split(mkv_file)
    file_name = file_mkv[0:len(file_mkv) - 4]
    extract_folder = os.path.join(folder_name + "/" + file_name)
    !mkdir "{extract_folder}"
    clear_output()
    cmd = ["mkvmerge --identify --identification-format json " + '"' + mkv_file + '"']
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    json_data, err = proc.communicate()
    json_data = json_data.decode("utf-8")
    json_data = json.loads(json_data)

    track_count = len(json_data['tracks'])

    extract_track_command = []

    for i in range(track_count):
        id = json_data.get('tracks')[int(i)].get('id')
        codec_id = json_data.get('tracks')[int(i)].get('properties').get('codec_id')
        language = json_data.get('tracks')[int(i)].get('properties').get('language')
        language_ietf = json_data.get('tracks')[int(i)].get('properties').get('language_ietf')
        title = json_data.get('tracks')[int(i)].get('properties').get('track_name')
        codec = json_data.get('tracks')[int(i)].get('codec')
        track_type = json_data.get('tracks')[int(i)].get('type')

        process_mkv_file()
        extract_track_param = (f'{str(id)}:"{str(extract_folder)}/{str(extract_track_name)}"')
        extract_track_command.append(extract_track_param)
        extract_param_option = ' '.join(extract_track_command)
    command = (f'mkvextract "{os.path.abspath(mkv_file)}" tracks {extract_param_option}')
    print("Extracting....")
    subprocess.call(command, shell=True, stdout=subprocess.PIPE)
    clear_output()
    print("Successfully Extracted.")

def execute_command_single_track():
    global id
    global codec_id
    global language
    global language_ietf
    global title
    global codec
    global track_type

    folder_name = os.path.dirname(os.path.abspath(mkv_file))
    folder_mkv, file_mkv = os.path.split(mkv_file)
    file_name = file_mkv[0:len(file_mkv) - 4]
    extract_folder = os.path.join(folder_name + "/" + file_name)
    !mkdir "{extract_folder}"
    clear_output()
    !mkvmerge -i "{mkv_file}"
    cmd = ["mkvmerge --identify --identification-format json " + '"' + mkv_file + '"']
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    json_data, err = proc.communicate()
    json_data = json_data.decode("utf-8")
    json_data = json.loads(json_data)

    i = input("Enter track ID: ")
    id = json_data.get('tracks')[int(i)].get('id')
    codec_id = json_data.get('tracks')[int(i)].get('properties').get('codec_id')
    language = json_data.get('tracks')[int(i)].get('properties').get('language')
    language_ietf = json_data.get('tracks')[int(i)].get('properties').get('language_ietf')
    title = json_data.get('tracks')[int(i)].get('properties').get('track_name')
    codec = json_data.get('tracks')[int(i)].get('codec')
    track_type = json_data.get('tracks')[int(i)].get('type')
    process_mkv_file()
    extract_track_param = (f'{str(id)}:"{str(extract_folder)}/{str(extract_track_name)}"')
    command = (f'mkvextract "{os.path.abspath(mkv_file)}" tracks {extract_track_param}')
    print("Extracting....")
    subprocess.call(command, shell=True, stdout=subprocess.PIPE)
    clear_output()
    print("Successfully Extracted.")

def execute_command_attachments():
    folder_name = os.path.dirname(os.path.abspath(mkv_file))
    folder_mkv, file_mkv = os.path.split(mkv_file)
    file_name = file_mkv[0:len(file_mkv) - 4]
    extract_folder = os.path.join(folder_name + "/" + file_name)
    !mkdir "{extract_folder}"
    clear_output()
    cmd = ["mkvmerge --identify --identification-format json " + '"' + mkv_file + '"']
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    json_data, err = proc.communicate()
    json_data = json_data.decode("utf-8")
    json_data = json.loads(json_data)
    if not (json_data.get("attachments") is None):
        for attachment in json_data["attachments"]:
            id_attachments = attachment["id"]
            filename_attachments = attachment["file_name"]
            extract_track_param = (f'{str(id_attachments)}:"{str(extract_folder)}/{str(filename_attachments)}"')
            command = (f'mkvextract "{os.path.abspath(mkv_file)}" attachments {extract_track_param}')
            print("Extracting....")
            subprocess.call(command, shell=True, stdout=subprocess.PIPE)
            clear_output()
            print("Successfully Extracted.")
    else:
        print("No attachments are available.")

def execute_command_chapters():
    folder_name = os.path.dirname(os.path.abspath(mkv_file))
    folder_mkv, file_mkv = os.path.split(mkv_file)
    file_name = file_mkv[0:len(file_mkv) - 4]
    extract_folder = os.path.join(folder_name + "/" + file_name)
    !mkdir "{extract_folder}"
    clear_output()
    cmd = ["mkvmerge --identify --identification-format json " + '"' + mkv_file + '"']
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    json_data, err = proc.communicate()
    json_data = json_data.decode("utf-8")
    json_data = json.loads(json_data)
    if not (json_data.get("chapters") is None):
        extract_track_param = (f'"{str(extract_folder)}/chapters.xml"')
        command = (f'mkvextract "{os.path.abspath(mkv_file)}" chapters {extract_track_param}')
        print("Extracting....")
        subprocess.call(command, shell=True, stdout=subprocess.PIPE)
        clear_output()
        print("Successfully Extracted.")
    else:
        print("No chapters are available.")

def execute_command_chapters_OGM():
    folder_name = os.path.dirname(os.path.abspath(mkv_file))
    folder_mkv, file_mkv = os.path.split(mkv_file)
    file_name = file_mkv[0:len(file_mkv) - 4]
    extract_folder = os.path.join(folder_name + "/" + file_name)
    !mkdir "{extract_folder}"
    clear_output()
    cmd = ["mkvmerge --identify --identification-format json " + '"' + mkv_file + '"']
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    json_data, err = proc.communicate()
    json_data = json_data.decode("utf-8")
    json_data = json.loads(json_data)
    if not (json_data.get("chapters") is None):
        extract_track_param = (f'"{str(extract_folder)}/chapters_OGM.txt"')
        command = (f'mkvextract "{os.path.abspath(mkv_file)}" chapters --simple {extract_track_param}')
        print("Extracting....")
        subprocess.call(command, shell=True, stdout=subprocess.PIPE)
        clear_output()
        print("Successfully Extracted.")
    else:
        print("No chapters are available.")

def execute_command_tags():
    folder_name = os.path.dirname(os.path.abspath(mkv_file))
    folder_mkv, file_mkv = os.path.split(mkv_file)
    file_name = file_mkv[0:len(file_mkv) - 4]
    extract_folder = os.path.join(folder_name + "/" + file_name)
    !mkdir "{extract_folder}"
    clear_output()
    cmd = ["mkvmerge --identify --identification-format json " + '"' + mkv_file + '"']
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    json_data, err = proc.communicate()
    json_data = json_data.decode("utf-8")
    json_data = json.loads(json_data)
    if not (json_data.get("global_tags") is None):
        extract_track_param = (f'"{str(extract_folder)}/tags.xml"')
        command = (f'mkvextract "{os.path.abspath(mkv_file)}" tags {extract_track_param}')
        print("Extracting....")
        subprocess.call(command, shell=True, stdout=subprocess.PIPE)
        clear_output()
        print("Successfully Extracted.")
    else:
        print("No tags are available.")

def execute_command_timecodes():
    folder_name = os.path.dirname(os.path.abspath(mkv_file))
    folder_mkv, file_mkv = os.path.split(mkv_file)
    file_name = file_mkv[0:len(file_mkv) - 4]
    extract_folder = os.path.join(folder_name + "/" + file_name)
    !mkdir "{extract_folder}"
    clear_output()
    cmd = ["mkvmerge --identify --identification-format json " + '"' + mkv_file + '"']
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    json_data, err = proc.communicate()
    json_data = json_data.decode("utf-8")
    json_data = json.loads(json_data)

    track_count = len(json_data['tracks'])

    extract_track_command = []

    for i in range(track_count):
        id = json_data.get('tracks')[int(i)].get('id')
        track_type = json_data.get('tracks')[int(i)].get('type')
        extract_track_name = str(id) + "_" + str(track_type) + "_tc.txt"
        extract_track_param = (f'{str(id)}:"{str(extract_folder)}/{str(extract_track_name)}"')
        extract_track_command.append(extract_track_param)
        extract_param_option = ' '.join(extract_track_command)
    command = (f'mkvextract "{os.path.abspath(mkv_file)}" timecodes_v2 {extract_param_option}')
    print("Extracting....")
    subprocess.call(command, shell=True, stdout=subprocess.PIPE)
    clear_output()
    print("Successfully Extracted.")

def execute_command_cues():
    folder_name = os.path.dirname(os.path.abspath(mkv_file))
    folder_mkv, file_mkv = os.path.split(mkv_file)
    file_name = file_mkv[0:len(file_mkv) - 4]
    extract_folder = os.path.join(folder_name + "/" + file_name)
    !mkdir "{extract_folder}"
    clear_output()
    cmd = ["mkvmerge --identify --identification-format json " + '"' + mkv_file + '"']
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    json_data, err = proc.communicate()
    json_data = json_data.decode("utf-8")
    json_data = json.loads(json_data)

    track_count = len(json_data['tracks'])

    extract_track_command = []

    for i in range(track_count):
        id = json_data.get('tracks')[int(i)].get('id')
        track_type = json_data.get('tracks')[int(i)].get('type')
        extract_track_name = str(id) + "_" + str(track_type) + "_cues.txt"
        extract_track_param = (f'{str(id)}:"{str(extract_folder)}/{str(extract_track_name)}"')
        extract_track_command.append(extract_track_param)
        extract_param_option = ' '.join(extract_track_command)
    command = (f'mkvextract "{os.path.abspath(mkv_file)}" cues {extract_param_option}')
    print("Extracting....")
    subprocess.call(command, shell=True, stdout=subprocess.PIPE)
    clear_output()
    print("Successfully Extracted.")

def execute_command_cue_sheet():
    folder_name = os.path.dirname(os.path.abspath(mkv_file))
    folder_mkv, file_mkv = os.path.split(mkv_file)
    file_name = file_mkv[0:len(file_mkv) - 4]
    extract_folder = os.path.join(folder_name + "/" + file_name)
    !mkdir "{extract_folder}"
    clear_output()
    extract_track_param = (f'"{str(extract_folder)}/cue_sheet.cue"')
    command = (f'mkvextract "{os.path.abspath(mkv_file)}" cuesheet {extract_track_param}')
    print("Extracting....")
    subprocess.call(command, shell=True, stdout=subprocess.PIPE)
    clear_output()
    print("Successfully Extracted.")

if extract_mode == "Tracks (All)":
  execute_command_tracks()
elif extract_mode == "Single Track":
  execute_command_single_track()
elif extract_mode == "Chapters":
  if chapters_mode == "XML":
    execute_command_chapters()
  elif chapters_mode == "OGM":
    execute_command_chapters_OGM()  
elif extract_mode == "Attachments":
  execute_command_attachments()
elif extract_mode == "Tags":
  execute_command_tags()
elif extract_mode == "Timecodes":
  execute_command_timecodes()
elif extract_mode == "Cues":
  execute_command_cues()
elif extract_mode == "Cue Sheet":
  execute_command_cue_sheet()

### __mkvpropedit__

* You can edit **segment info, track info, chapters, attachments, tags** in a mkv file with below cell.
* If you want to **delete statistic tags** from tracks select `delete_track_statistics_tags`

In [ ]:
import os
import sys
import subprocess
import json
from IPython.display import clear_output

mkv_file = "" #@param {type:"string"}
handle_mode = "" #@param ["", "Segment Info", "Tracks", "Chapters", "Attachments", "Tags"]
delete_track_statistics_tags = False #@param {type:"boolean"}

if delete_track_statistics_tags == True:
  !mkvpropedit "{mkv_file}" --delete-track-statistics-tags

def edit_track_properties():
    global id
    global codec_id
    global language
    global language_ietf
    global title
    global codec
    global track_type

    cmd = ["mkvmerge --identify --identification-format json " + '"' + mkv_file + '"']
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    json_data, err = proc.communicate()
    json_data = json_data.decode("utf-8")
    json_data = json.loads(json_data)

    track_count = len(json_data['tracks'])

    for i in range(track_count):
        id = json_data.get('tracks')[int(i)].get('id')
        codec_id = json_data.get('tracks')[int(i)].get('properties').get('codec_id')
        language = json_data.get('tracks')[int(i)].get('properties').get('language')
        language_ietf = json_data.get('tracks')[int(i)].get('properties').get('language_ietf')
        title = json_data.get('tracks')[int(i)].get('properties').get('track_name')
        codec = json_data.get('tracks')[int(i)].get('codec')
        track_type = json_data.get('tracks')[int(i)].get('type')

        print("Track ID " + str(id) + " : " + str(language) + " | " + str(codec))

    print(" ")
    print("Edit track properties")
    print(" ")
    track_id = int(input("Enter Track ID: "))
    mkvpropedit_track_id = track_id + 1
    print(" ")
    print("Available options")
    print("1 : Track name")
    print("2 : Track language")
    print("3 : Track default flag")
    print("4 : Track forced flag")
    prop_option = int(input("Enter option number: "))
    if prop_option == 1:
      print(" ")
      print("1 : add a track name")
      print("2 : change the track name")
      print("3 : delete the track name")
      track_opt = int(input("Enter number: "))
      print(" ")
      if track_opt == 1:
        track_name = input("Enter track name: ")
        command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --edit track:{mkvpropedit_track_id} --add name="{track_name}"')
      elif track_opt == 2:
        track_name = input("Enter track name: ")
        command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --edit track:{mkvpropedit_track_id} --set name="{track_name}"')
      elif track_opt == 3:
        command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --edit track:{mkvpropedit_track_id} --delete name')
    elif prop_option == 2:
      print(" ")
      print("1 : add a track language")
      print("2 : change the track language")
      print("3 : delete the track language")
      track_opt = int(input("Enter number: "))
      print(" ")
      if track_opt == 1:
        track_lang = input("Enter track language: ")
        command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --edit track:{mkvpropedit_track_id} --add language="{track_lang}"')
      elif track_opt == 2:
        track_lang = input("Enter track language: ")
        command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --edit track:{mkvpropedit_track_id} --set language="{track_lang}"')
      elif track_opt == 3:
        command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --edit track:{mkvpropedit_track_id} --delete language')
    elif prop_option == 3:
      print(" ")
      print("0 : No")
      print("1 : Yes")
      default_track = input("Track default: ")
      command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --edit track:{mkvpropedit_track_id} --set flag-default={default_track}')
    elif prop_option == 4:
      print(" ")
      print("0 : No")
      print("1 : Yes")
      forced_track = input("Track forced: ")
      command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --edit track:{mkvpropedit_track_id} --set flag-forced={forced_track}')
        
    
    subprocess.call(command, shell=True, stdout=subprocess.PIPE)
    clear_output()
    print("Successfully Edited.")

def edit_segment_properties():
    print("Segment info")
    print("1 : add a title")
    print("2 : change the title")
    print("3 : delete the title")
    track_opt = int(input("Enter number: "))
    print(" ")
    if track_opt == 1:
      segment_title = input("Enter title: ")
      command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --edit info --add title="{segment_title}"')
    elif track_opt == 2:
      segment_title = input("Enter title: ")
      command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --edit info --set title="{segment_title}"')
    elif track_opt == 3:
      command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --edit info --delete title')
    subprocess.call(command, shell=True, stdout=subprocess.PIPE)
    clear_output()
    print("Successfully Edited.")

def edit_chapter_properties():
    print("Chapters")
    print("1 : add chapters")
    print("2 : remove chapters")
    track_opt = int(input("Enter number: "))
    print(" ")
    if track_opt == 1:
      chapters_xml = input("Enter XML file path: ")
      command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --chapters "{chapters_xml}"')
    elif track_opt == 2:
      command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --chapters ""')
    subprocess.call(command, shell=True, stdout=subprocess.PIPE)
    clear_output()
    print("Successfully Edited.")

def list_attachments():
    global attach_id
    global attach_file_name
    global content_type
    global description
    global uid
    global json_data

    cmd = ["mkvmerge --identify --identification-format json " + '"' + mkv_file + '"']
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    json_data, err = proc.communicate()
    json_data = json_data.decode("utf-8")
    json_data = json.loads(json_data)

    attachment_count = len(json_data['attachments'])

    for i in range(attachment_count):
        attach_id = json_data.get('attachments')[int(i)].get('id')
        attach_file_name = json_data.get('attachments')[int(i)].get('file_name')
        content_type = json_data.get('attachments')[int(i)].get('content_type')
        description = json_data.get('attachments')[int(i)].get('description')
        uid = json_data.get('attachments')[int(i)].get('properties').get('uid')

        print("Attachment ID : " + str(attach_id))
        print("|")
        print("|-- file name : " + str(attach_file_name))
        print("|-- MIME type : " + str(content_type))
        print("|-- description : " + str(description))
        print("|-- UID : " + str(uid))
        print(" ")

def edit_attachment_properties():
    print("Attachments")
    print("1 : Add attachment")
    print("2 : Replace attachment")
    print("3 : Update attachment")
    print("4 : Delete attachment")
    print("5 : List attachments")
    track_opt = int(input("Enter number: "))
    print(" ")
    if track_opt == 1:
      attach_file = input("Enter attachment file path: ")
      folder_attach, file_attach = os.path.split(attach_file)
      ext_list = file_attach.split('.')
      attach_ext = ext_list[-1]
      print(" ")
      print("Set attachment properties or leave it blank")
      print(" ")
      attach_name = input("Add an attachment name: ")
      attach_mime = input("Add an attachment MIME-type: ")
      attach_desc = input("Add an attachment description: ")
      attach_uid = input("Add an attachment UID: ")
      if attach_name == "":
        if attach_mime == "":
          if attach_desc == "":
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --add-attachment "{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-uid "{attach_uid}" --add-attachment "{attach_file}"')
          else:
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --add-attachment "{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-uid "{attach_uid}" --add-attachment "{attach_file}"')
        else:
          if attach_desc == "":
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-mime-type "{attach_mime}" --add-attachment "{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-uid "{attach_uid}" --attachment-mime-type "{attach_mime}" --add-attachment "{attach_file}"')
          else:
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-mime-type "{attach_mime}" --add-attachment "{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-uid "{attach_uid}" --attachment-mime-type "{attach_mime}" --add-attachment "{attach_file}"')
      else:
        if attach_mime == "":
          if attach_desc == "":
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-name "{attach_name}".{attach_ext} --add-attachment "{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-uid "{attach_uid}" --attachment-name "{attach_name}".{attach_ext} --add-attachment "{attach_file}"')
          else:
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-name "{attach_name}".{attach_ext} --add-attachment "{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-uid "{attach_uid}" --attachment-name "{attach_name}".{attach_ext} --add-attachment "{attach_file}"')
        else:
          if attach_desc == "":
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-mime-type "{attach_mime}" --attachment-name "{attach_name}".{attach_ext} --add-attachment "{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-uid "{attach_uid}" --attachment-mime-type "{attach_mime}" --attachment-name "{attach_name}".{attach_ext} --add-attachment "{attach_file}"')
          else:
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-mime-type "{attach_mime}" --attachment-name "{attach_name}".{attach_ext} --add-attachment "{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-uid "{attach_uid}" --attachment-mime-type "{attach_mime}" --attachment-name "{attach_name}".{attach_ext} --add-attachment "{attach_file}"')
      subprocess.call(command, shell=True, stdout=subprocess.PIPE)
      clear_output()
      print("Successfully Edited.")
    if track_opt == 2:
      list_attachments()
      attachment_id = input("Enter attachment ID to replace: ")
      attach_file = input("Enter attachment file path to replace: ")
      folder_attach, file_attach = os.path.split(attach_file)
      ext_list = file_attach.split('.')
      attach_ext = ext_list[-1]
      print(" ")
      print("Set attachment properties or leave it blank")
      print(" ")
      attach_name = input("Add an attachment name: ")
      attach_mime = input("Add an attachment MIME-type: ")
      attach_desc = input("Add an attachment description: ")
      attach_uid = input("Add an attachment UID: ")
      if attach_name == "":
        if attach_mime == "":
          if attach_desc == "":
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --replace-attachment {attachment_id}:"{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-uid "{attach_uid}" --replace-attachment {attachment_id}:"{attach_file}"')
          else:
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --replace-attachment {attachment_id}:"{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-uid "{attach_uid}" --replace-attachment {attachment_id}:"{attach_file}"')
        else:
          if attach_desc == "":
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-mime-type "{attach_mime}" --replace-attachment {attachment_id}:"{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-uid "{attach_uid}" --attachment-mime-type "{attach_mime}" --replace-attachment {attachment_id}:"{attach_file}"')
          else:
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-mime-type "{attach_mime}" --replace-attachment {attachment_id}:"{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-uid "{attach_uid}" --attachment-mime-type "{attach_mime}" --replace-attachment {attachment_id}:"{attach_file}"')
      else:
        if attach_mime == "":
          if attach_desc == "":
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-name "{attach_name}".{attach_ext} --replace-attachment {attachment_id}:"{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-uid "{attach_uid}" --attachment-name "{attach_name}".{attach_ext} --replace-attachment {attachment_id}:"{attach_file}"')
          else:
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-name "{attach_name}".{attach_ext} --replace-attachment {attachment_id}:"{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-uid "{attach_uid}" --attachment-name "{attach_name}".{attach_ext} --replace-attachment {attachment_id}:"{attach_file}"')
        else:
          if attach_desc == "":
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-mime-type "{attach_mime}" --attachment-name "{attach_name}".{attach_ext} --replace-attachment {attachment_id}:"{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-uid "{attach_uid}" --attachment-mime-type "{attach_mime}" --attachment-name "{attach_name}".{attach_ext} --replace-attachment {attachment_id}:"{attach_file}"')
          else:
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-mime-type "{attach_mime}" --attachment-name "{attach_name}".{attach_ext} --replace-attachment {attachment_id}:"{attach_file}"')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-uid "{attach_uid}" --attachment-mime-type "{attach_mime}" --attachment-name "{attach_name}".{attach_ext} --replace-attachment {attachment_id}:"{attach_file}"')
      subprocess.call(command, shell=True, stdout=subprocess.PIPE)
      clear_output()
      print("Successfully Edited.")   
    if track_opt == 3:
      list_attachments()
      attachment_id = input("Enter attachment ID to update: ")
      att_id = int(attachment_id) - 1
      att_file_name = json_data.get('attachments')[int(att_id)].get('file_name')
      ext_list = att_file_name.split('.')
      attach_ext = ext_list[-1]
      print(" ")
      print("Update attachment properties or leave it blank")
      print(" ")
      attach_name = input("Update attachment name: ")
      attach_mime = input("Update attachment MIME-type: ")
      attach_desc = input("Update attachment description: ")
      attach_uid = input("Update attachment UID: ")
      if attach_name == "":
        if attach_mime == "":
          if attach_desc == "":
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --update-attachment {attachment_id}')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-uid "{attach_uid}" --update-attachment {attachment_id}')
          else:
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --update-attachment {attachment_id}')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-uid "{attach_uid}" --update-attachment {attachment_id}')
        else:
          if attach_desc == "":
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-mime-type "{attach_mime}" --update-attachment {attachment_id}')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-uid "{attach_uid}" --attachment-mime-type "{attach_mime}" --update-attachment {attachment_id}')
          else:
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-mime-type "{attach_mime}" --update-attachment {attachment_id}')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-uid "{attach_uid}" --attachment-mime-type "{attach_mime}" --update-attachment {attachment_id}')
      else:
        if attach_mime == "":
          if attach_desc == "":
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-name "{attach_name}".{attach_ext} --update-attachment {attachment_id}')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-uid "{attach_uid}" --attachment-name "{attach_name}".{attach_ext} --update-attachment {attachment_id}')
          else:
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-name "{attach_name}".{attach_ext} --update-attachment {attachment_id}')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-uid "{attach_uid}" --attachment-name "{attach_name}".{attach_ext} --update-attachment {attachment_id}')
        else:
          if attach_desc == "":
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-mime-type "{attach_mime}" --attachment-name "{attach_name}".{attach_ext} --update-attachment {attachment_id}')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-uid "{attach_uid}" --attachment-mime-type "{attach_mime}" --attachment-name "{attach_name}".{attach_ext} --update-attachment {attachment_id}')
          else:
            if attach_uid == "":
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-mime-type "{attach_mime}" --attachment-name "{attach_name}".{attach_ext} --update-attachment {attachment_id}')
            else:
              command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --attachment-description "{attach_desc}" --attachment-uid "{attach_uid}" --attachment-mime-type "{attach_mime}" --attachment-name "{attach_name}".{attach_ext} --update-attachment {attachment_id}')
      subprocess.call(command, shell=True, stdout=subprocess.PIPE)
      clear_output()
      print("Successfully Edited.")    
    if track_opt == 4:
      list_attachments()
      attachment_id = input("Enter attachment ID to delete: ")
      command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --delete-attachment {attachment_id}')
      subprocess.call(command, shell=True, stdout=subprocess.PIPE)
      clear_output()
      print("Successfully Edited.")
    if track_opt == 5:
      clear_output()
      list_attachments()

def edit_tag_properties():
    global id
    global codec_id
    global language
    global language_ietf
    global title
    global codec
    global track_type

    print("Tags")
    print("1 : All tags")
    print("2 : Global tags")
    print("3 : Track tags")
    tags_opt = int(input("Enter number: "))
    if tags_opt == 1:
      print(" ")
      print("1 : Add tags")
      print("2 : Remove tags")
      tags_opt_v2 = int(input("Enter number: "))
      if tags_opt_v2 == 1:
        print(" ")
        tags_xml = input("Enter XML file path: ")
        command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --tags all:"{tags_xml}"')
      elif tags_opt_v2 == 2:
        command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --tags all:')
    elif tags_opt == 2:
      print(" ")
      print("1 : Add global tags")
      print("2 : Remove global tags")
      tags_opt_v2 = int(input("Enter number: "))
      if tags_opt_v2 == 1:
        print(" ")
        tags_xml = input("Enter XML file path: ")
        command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --tags global:"{tags_xml}"')
      elif tags_opt_v2 == 2:
        command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --tags global:')
    elif tags_opt == 3:
      cmd = ["mkvmerge --identify --identification-format json " + '"' + mkv_file + '"']
      proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
      json_data, err = proc.communicate()
      json_data = json_data.decode("utf-8")
      json_data = json.loads(json_data)

      track_count = len(json_data['tracks'])

      print(" ")
      for i in range(track_count):
          id = json_data.get('tracks')[int(i)].get('id')
          codec_id = json_data.get('tracks')[int(i)].get('properties').get('codec_id')
          language = json_data.get('tracks')[int(i)].get('properties').get('language')
          language_ietf = json_data.get('tracks')[int(i)].get('properties').get('language_ietf')
          title = json_data.get('tracks')[int(i)].get('properties').get('track_name')
          codec = json_data.get('tracks')[int(i)].get('codec')
          track_type = json_data.get('tracks')[int(i)].get('type')

          print("Track ID " + str(id) + " : " + str(language) + " | " + str(codec))

      print(" ")
      track_id = int(input("Enter Track ID: "))
      mkvpropedit_track_id = track_id + 1
      print(" ")
      print("1 : Add track tags")
      print("2 : Remove track tags")
      tags_opt_v2 = int(input("Enter number: "))
      if tags_opt_v2 == 1:
        print(" ")
        tags_xml = input("Enter XML file path: ")
        command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --tags track:{mkvpropedit_track_id}:"{tags_xml}"')
      elif tags_opt_v2 == 2:
        command = (f'mkvpropedit "{os.path.abspath(mkv_file)}" --tags track:{mkvpropedit_track_id}:')
    subprocess.call(command, shell=True, stdout=subprocess.PIPE)
    clear_output()
    print("Successfully Edited.")

if handle_mode == "Tracks":
  edit_track_properties()
elif handle_mode == "Segment Info":
  edit_segment_properties()
elif handle_mode == "Chapters":
  edit_chapter_properties()
elif handle_mode == "Attachments":
  edit_attachment_properties()
elif handle_mode == "Tags":
  edit_tag_properties()